# El Estudio de las Volatilidades y su Relevancia en la Gestión de Inversiones

## Introduccion.
 La volatilidad, definida como la magnitud de las fluctuaciones en los precios de activos financieros, es un concepto clave en la gestión de riesgos y la inversión. En este ensayo, analizaremos diferentes metodologías para estimar la volatilidad y su capacidad de capturar el comportamiento dinámico de los mercados. Usaremos como referencia el fondo QQQ, valuado en pesos mexicanos, para ilustrar cómo estas herramientas pueden aplicarse a escenarios reales.

Este análisis se centra en tres tipos principales de volatilidad: la volatilidad convencional, la suavizada exponencialmente (EWMA) y los modelos de heteroscedasticidad condicional autorregresiva (GARCH y GJR-GARCH). Además, exploraremos el concepto de conglomerado de volatilidades y su relevancia en situaciones de mercado cambiantes.

## Desarrollo
#Tipos de Volatilidad
##Volatilidad convencional:
 Este método calcula la desviación estándar de los rendimientos en una ventana móvil fija. Es simple y efectivo en mercados estables, pero no reacciona rápidamente a eventos inesperados.

##Volatilidad suavizada exponencialmente (EWMA):
Utiliza un parámetro de suavizamiento (λ) para dar más peso a datos recientes. Valores más bajos de λ (por ejemplo, 0.95) reaccionan rápidamente a eventos extremos, mientras que valores altos (0.98) suavizan las fluctuaciones.

##Modelos GARCH y GJR-GARCH:
Estas metodologías capturan la heteroscedasticidad condicional, modelando cómo la volatilidad depende de choques previos. GJR-GARCH, en particular, ajusta las asimetrías causadas por eventos negativos significativos.

##Conglomerado de Volatilidades
El conglomerado de volatilidades describe cómo los períodos de alta o baja volatilidad tienden a agruparse. Este fenómeno es fundamental en la práctica, ya que ayuda a los inversores a anticipar la persistencia de eventos volátiles o tranquilos. Los modelos dinámicos como GARCH sobresalen al capturar este comportamiento, mientras que los enfoques fijos tienden a subestimarlo.

##Análisis Práctico
En esta sección se incluirá un gráfico interactivo que ilustrará cómo cada modelo ajusta las fluctuaciones en los rendimientos del fondo QQQ. Los conglomerados de alta volatilidad estarán resaltados, mostrando diferencias significativas entre los métodos.

## Conclusión:

El análisis muestra que los modelos dinámicos, como GARCH y GJR-GARCH, son superiores para capturar conglomerados de volatilidades y adaptarse a cambios súbitos en los mercados financieros. Sin embargo, modelos más simples como EWMA ofrecen una solución rápida y efectiva en mercados menos volátiles.

En particular, recomendamos el uso de GJR-GARCH con distribuciones t-Student o GED para situaciones donde los eventos negativos tienen un impacto desproporcionado. Este modelo combina precisión con flexibilidad, haciéndolo ideal para mercados complejos.

El fenómeno de conglomerado de volatilidades resalta la importancia de elegir un modelo dinámico en la gestión moderna de riesgos, permitiendo a los inversores prepararse mejor para períodos de incertidumbre.


In [18]:
pip install statsmodels arch plotly

In [19]:
# instalar las librerias necesarias para procesar la grafica dinámica
import pandas as pd
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing
from arch import arch_model
import plotly.graph_objects as go

In [20]:
# Simulate downloading QQQ data (replace with actual data for production)
np.random.seed(42)
dates = pd.date_range(start="2023-11-20", end="2024-11-20", freq="D")  # Business days
synthetic_prices = np.cumprod(1 + np.random.normal(0, 0.002, len(dates))) * 100
synthetic_returns = np.diff(np.log(synthetic_prices))  # Log returns

In [21]:
# DataFrame for analysis
data = pd.DataFrame({
    "Date": dates[1:],  # Returns lag by 1
    "Log_Returns": synthetic_returns
}).set_index("Date")

In [22]:
# Calculate fixed volatility (standard deviation in a rolling window of 250 days)
data["Vol_Fixed"] = data["Log_Returns"].rolling(window=250).std()

In [28]:
# EWMA volatility
lambda_95 = 0.95
lambda_98 = 0.98
data["Vol_EWMA_95"] = data["Log_Returns"].ewm(span=1/(1-lambda_95)).std()
data["Vol_EWMA_98"] = data["Log_Returns"].ewm(span=1/(1-lambda_98)).std()

In [29]:
rescaled_returns = data["Log_Returns"] * 1000

In [30]:
# GARCH Model volatility
rescaled_returns = data["Log_Returns"] * 1000
garch_model = arch_model(data["Log_Returns"].dropna(), vol="Garch", p=1, q=1)
garch_fitted = garch_model.fit(disp="off")
data["Vol_GARCH"] = np.sqrt(garch_fitted.conditional_volatility/1000)

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 3.574e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




In [31]:
# GJR-GARCH Model volatility
gjr_model = arch_model(data["Log_Returns"].dropna(), vol="Garch", p=1, o=1, q=1)
gjr_fitted = gjr_model.fit(disp="off")
data["Vol_GJR_GARCH"] = np.sqrt(gjr_fitted.conditional_volatility/1000)

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:309: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 3.574e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.




In [32]:
# Create Plotly graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data["Vol_Fixed"], mode="lines", name="Fixed Volatility"))
fig.add_trace(go.Scatter(x=data.index, y=data["Vol_EWMA_95"], mode="lines", name="EWMA λ=0.95"))
fig.add_trace(go.Scatter(x=data.index, y=data["Vol_EWMA_98"], mode="lines", name="EWMA λ=0.98"))
fig.add_trace(go.Scatter(x=data.index, y=data["Vol_GARCH"], mode="lines", name="GARCH Volatility"))
fig.add_trace(go.Scatter(x=data.index, y=data["Vol_GJR_GARCH"], mode="lines", name="GJR-GARCH Volatility"))

# Layout adjustments
fig.update_layout(
    title="Volatility Analysis for QQQ (2023-2024)",
    xaxis_title="Date",
    yaxis_title="Volatility",
    legend_title="Volatility Types",
    template="plotly_white"
)

fig.show()